In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

memory.save_context({"input": "HI!"}, {"output": "How are you?"})

memory.load_memory_variables({})

In [3]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
  return_messages=True,
  k=4
)

def add_message(input, output):
  memory.save_context({"input": input}, {"output": output})
  
add_message("HI!", "How are you?")
add_message("I'm good, how are you?", "I'm good too!")
add_message("That's great!", "Yes it is!")
add_message("I'm glad you're doing well", "Thank you!")

In [4]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='HI!'),
  AIMessage(content='How are you?'),
  HumanMessage(content="I'm good, how are you?"),
  AIMessage(content="I'm good too!"),
  HumanMessage(content="That's great!"),
  AIMessage(content='Yes it is!'),
  HumanMessage(content="I'm glad you're doing well"),
  AIMessage(content='Thank you!')]}

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [ ]:
add_message("South Kddorea is so pretty", "I wish I could go!!!")

In [ ]:
get_history()

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [ ]:
get_history() 
# {'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
# AIMessage(content='Wow that is so cool!')]}

In [ ]:
add_message("South Korea is so pretty", "I wish I could go!!!")

In [ ]:
get_history()
#{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
# AIMessage(content='Wow that is so cool!'),
# HumanMessage(content='South Korea is so pretty'),
# AIMessage(content='I wish I could go!!!'),
# HumanMessage(content='How far is Korea from Argentina?'),
# AIMessage(content="I don't know! Super far!")]}

In [ ]:
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")

In [ ]:
get_history()
  #{'history': [SystemMessage(content='The human introduces themselves as Nicolas and mentions that they live in South Korea. The AI responds by expressing excitement and finding it cool.'),
  #HumanMessage(content='South Korea is so pretty'),
  #AIMessage(content='I wish I could go!!!'),
  #HumanMessage(content='How far is Korea from Argentina?'),
  #AIMessage(content="I don't know! Super far!"),
  #HumanMessage(content='How far is Brazil from Argentina?'),
  #AIMessage(content="I don't know! Super far!"),
  #HumanMessage(content='How far is Brazil from Argentina?'),
  #AIMessage(content="I don't know! Super far!"),
  #HumanMessage(content='How far is Brazil from Argentina?'),
  #AIMessage(content="I don't know! Super far!"),
  #HumanMessage(content='How far is Brazil from Argentina?'),
  #AIMessage(content="I don't know! Super far!")]}

In [ ]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [ ]:
memory.load_memory_variables({"input": "who is Nicolas"})
#{'history': [SystemMessage(content='On Nicolas: Nicolas is a person. Nicolas lives in South Korea.')]}

In [ ]:
add_message("Nicolas likes kimchi", "Wow that is so cool!")

In [ ]:
memory.load_memory_variables({"inputs": "what does nicolas like"})
#{'history': [SystemMessage(content='On Nicolas: Nicolas is a person. Nicolas lives in South Korea. Nicolas likes kimchi.')]}

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Nico")

In [ ]:
chain.predict(question="I live in Seoul")

In [ ]:
chain.predict(question="What is my name?")

In [7]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    print("test",RunnablePassthrough.assign(type(history=load_memory)))
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)
    print("메모리",type(memory.load_memory_variables({})["history"]))
    

In [8]:
invoke_chain("My name is nico")

test <class 'langchain.schema.runnable.passthrough.RunnableAssign'>
content='Nice to meet you, Nico! How can I assist you today?'
메모리 <class 'list'>


In [ ]:
invoke_chain("What is my name?")